In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

prompt_model_path = "D:\\models\\SportsFanBotGhostGPT"
tokenizer = GPT2Tokenizer.from_pretrained(prompt_model_path)
model = GPT2LMHeadModel.from_pretrained(prompt_model_path)

In [27]:
def validate_message(message):
	import re
	start_end_regex = re.compile("<\|startoftext\|>(.+?)<\|endoftext\|>")
	prompt_regex = re.compile("<\|prompt\|>(.+?)<\|text\|>")
	text_regex = re.compile("<\|text\|>(.+?)<\|endoftext\|>")
	found_start_end = start_end_regex.findall(message)
	if len(found_start_end) == 0:
		return "", ""

	generated_prompt = ""
	generated_text = ""

	found_prompt = prompt_regex.findall(message)
	if len(found_prompt) > 0:
		generated_prompt = found_prompt[0]

	found_text = text_regex.findall(message)
	if len(found_text) > 0:
		generated_text = found_text[0]


	return generated_prompt.strip(), generated_text.strip()

In [29]:
def generate_prompt(model_name):
	question = f"<|startoftext|> <|model|> {model_name} <|prompt|>"

	prompt = f"{question}"

	device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

	generation_prompt = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")

	model.to(device)

	generation_prompt.to(device)

	inputs = generation_prompt.input_ids

	attention_mask = generation_prompt['attention_mask']

	sample_outputs = model.generate(inputs=inputs,
									attention_mask=attention_mask,
									do_sample=True,
									max_length=1024,
									num_return_sequences=1,
									repetition_penalty=1.1)
	result = ""
	for i, sample_output in enumerate(sample_outputs):
		result = tokenizer.decode(sample_output, skip_special_tokens=False)
		cleaned_prompt_text_pair = validate_message(result)
		return cleaned_prompt_text_pair

In [31]:
text, prompt = generate_prompt("CityDiffusion")
print(text)
print(prompt)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Nike, Barcelona, Spain
A group of people are walking down the street in front.
